In [1]:
#!/usr/bin/env python3

import pandas as pd
import errno    
import os
import numpy as np
import math

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

'''
Author: Enrico Ceccolini
    TODO write the description
'''

#datadir = "/datasets/eurora_data/db/"
datadir = "/datasets/eurora_data/db1/"
infile_jobs_to_nodes = datadir + "job_nodes.csv"
infile_jobs = datadir + "jobs.csv"

# settings
interval_comment = "WholeData"
suffix = "_1min_"

infile_large_jobs = datadir + "CPUs/" + interval_comment + "/large_jobs_real_pow.csv"

#start_time = pd.to_datetime('2014-03-31')
#end_time = pd.to_datetime('2015-08-11')

large_jobs = pd.read_csv(infile_large_jobs, index_col=0)

In [2]:
large_jobs.head(1)

,job_id_string,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,time_req,exit_status,real_pow,runned_alone,real_pow_quality
0,498458.node129,parallel,2014-03-31 00:30:02,2014-03-31 00:30:02,2014-03-31 01:22:40,alupi000@node129.eurora.cineca.it,1,16,14,04:00,COMPLETED,0.0,True,0.0


In [3]:
large_jobs_with_pow = large_jobs[large_jobs['real_pow'] != 0]
print("we have {}/{} jobs with a real power consumption value".format(large_jobs_with_pow.shape[0], large_jobs.shape[0]))

we have 57658/71391 jobs with a real power consumption value


In [4]:
large_jobs_with_pow

,job_id_string,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,time_req,exit_status,real_pow,runned_alone,real_pow_quality
1,498459.node129,parallel,2014-03-31 00:36:51,2014-03-31 00:36:51,2014-03-31 00:40:13,framondo@node129.eurora.cineca.it,1,16,14,01:00,COMPLETED,219.421912,True,1.000000
2,498460.node129,parallel,2014-03-31 00:54:37,2014-03-31 00:54:38,2014-03-31 04:55:17,vcalzia0@node129.eurora.cineca.it,6,96,84,04:00,COMPLETED,1233.914406,True,0.884722
3,498461.node129,parallel,2014-03-31 01:00:01,2014-03-31 01:00:02,2014-03-31 05:01:18,alupi000@node129.eurora.cineca.it,1,16,14,04:00,COMPLETED,204.274744,True,0.629167
7,498465.node129,R472836,2014-03-31 04:00:17,2014-03-31 04:00:17,2014-03-31 07:50:11,a07smr01@node129.eurora.cineca.it,1,1,2,06:00,COMPLETED,17.288127,False,1.000000
8,498466.node129,R472836,2014-03-31 04:00:17,2014-03-31 04:00:18,2014-03-31 04:23:30,a07smr01@node129.eurora.cineca.it,11,176,154,06:00,COMPLETED,3872.022801,True,0.727273
9,498467.node129,R472836,2014-03-31 04:23:23,2014-03-31 04:23:30,2014-03-31 05:19:43,a07smr01@node036.eurora.cineca.it,11,176,154,06:00,COMPLETED,1685.771628,True,0.727273
10,498468.node129,R472836,2014-03-31 04:23:25,2014-03-31 04:23:25,2014-03-31 07:44:59,a07smr01@node036.eurora.cineca.it,1,8,2,06:00,COMPLETED,67.131914,False,1.000000
11,498469.node129,R472836,2014-03-31 04:23:25,2014-03-31 04:23:26,2014-03-31 09:46:08,a07smr01@node036.eurora.cineca.it,1,1,2,06:00,COMPLETED,20.950996,False,1.000000
12,498470.node129,R472836,2014-03-31 04:23:26,2014-03-31 04:23:27,2014-03-31 09:33:53,a07smr01@node036.eurora.cineca.it,1,1,2,06:00,COMPLETED,10.047165,False,1.000000
13,498471.node129,R472836,2014-03-31 04:23:27,2014-03-31 04:23:27,2014-03-31 07:45:05,a07smr01@node036.eurora.cineca.it,1,1,2,06:00,COMPLETED,8.370311,False,1.000000


In [5]:
ran_alone = large_jobs_with_pow[large_jobs_with_pow['runned_alone'] == True]
print("we have {}/{} jobs ran alone".format(ran_alone.shape[0], large_jobs_with_pow.shape[0]))

we have 37958/57658 jobs ran alone


In [6]:
ran_alone

,job_id_string,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,time_req,exit_status,real_pow,runned_alone,real_pow_quality
1,498459.node129,parallel,2014-03-31 00:36:51,2014-03-31 00:36:51,2014-03-31 00:40:13,framondo@node129.eurora.cineca.it,1,16,14,01:00,COMPLETED,219.421912,True,1.000000
2,498460.node129,parallel,2014-03-31 00:54:37,2014-03-31 00:54:38,2014-03-31 04:55:17,vcalzia0@node129.eurora.cineca.it,6,96,84,04:00,COMPLETED,1233.914406,True,0.884722
3,498461.node129,parallel,2014-03-31 01:00:01,2014-03-31 01:00:02,2014-03-31 05:01:18,alupi000@node129.eurora.cineca.it,1,16,14,04:00,COMPLETED,204.274744,True,0.629167
8,498466.node129,R472836,2014-03-31 04:00:17,2014-03-31 04:00:18,2014-03-31 04:23:30,a07smr01@node129.eurora.cineca.it,11,176,154,06:00,COMPLETED,3872.022801,True,0.727273
9,498467.node129,R472836,2014-03-31 04:23:23,2014-03-31 04:23:30,2014-03-31 05:19:43,a07smr01@node036.eurora.cineca.it,11,176,154,06:00,COMPLETED,1685.771628,True,0.727273
15,498473.node129,R472836,2014-03-31 05:19:40,2014-03-31 05:19:44,2014-03-31 07:02:43,a07smr01@node036.eurora.cineca.it,11,176,154,06:00,COMPLETED,4796.686081,True,0.727273
16,498474.node129,R472836,2014-03-31 05:19:41,2014-03-31 05:19:41,2014-03-31 05:27:46,a07smr01@node036.eurora.cineca.it,1,1,2,06:00,COMPLETED,10.288684,True,1.000000
17,498475.node129,R472836,2014-03-31 05:27:45,2014-03-31 07:02:43,2014-03-31 07:23:25,a07smr01@node035.eurora.cineca.it,4,64,56,06:00,COMPLETED,1502.050365,True,1.000000
21,498479.node129,R472836,2014-03-31 07:02:40,2014-03-31 07:23:25,2014-03-31 07:40:17,a07smr01@node036.eurora.cineca.it,11,176,154,06:00,COMPLETED,4730.514758,True,0.727273
22,498480.node129,R472836,2014-03-31 07:40:10,2014-03-31 07:40:17,2014-03-31 07:54:40,a07smr01@node036.eurora.cineca.it,2,32,28,06:00,COMPLETED,786.045653,True,1.000000
